In [1]:
# For now, I'm just focusing on delta hedging. After that is done, we make a system of equations
# to set delta and vega to 0 and solve with np.linalg.solve() (see examples/ex6sol for reference).
# %pip install pyfinance
# %pip install py_vollib
import pandas as pd
from pyfinance.options import BSM
from datetime import datetime, timedelta
import py_vollib
from py_vollib.black_scholes.implied_volatility import implied_volatility

df = pd.read_csv("../data/apple.csv")
df = df.drop(["Unnamed: 0"], axis=1)
df['date'] = pd.to_datetime(df['Date'])
df['T'] = (pd.to_datetime('2024-01-19') - df['date']).dt.days / 252 # T = time to expiration in years, 252 trading days
df = df.drop(["date"], axis=1)
df.head()

,Date,Underlying,C170,C175,C180,C185,C190,P170,P175,P180,P185,P190,T
0,2023-08-21,175.84,16.40,13.27,10.30,7.85,5.91,6.81,8.65,10.85,13.35,16.50,0.599206
1,2023-08-22,177.23,17.50,14.10,11.10,8.51,6.34,6.26,8.10,10.10,12.75,15.50,0.595238
2,2023-08-23,181.12,20.20,16.65,13.45,10.55,8.08,5.25,6.71,8.50,10.70,13.33,0.591270
3,2023-08-24,176.38,16.97,13.75,10.80,8.35,6.25,6.85,8.62,10.75,13.35,16.60,0.587302
4,2023-08-25,178.61,18.50,15.15,11.85,9.19,6.90,5.90,7.50,9.50,11.90,14.81,0.583333


The delta measures the rate of change of the security wrt. to the price of the underlying. Thus, the delta of the stock is one.
Vega measures the rate of change of the security wrt. to the volatility of the underlying. Again, the vega of the stock is one. In delta hedging, we short the option and hold a dynamically adjusted amount of the risky asset to eliminate the risk.

Next, calculate the hedge with 2 portfolios: OP and RE. OP has a long call and RE has a short amount of the underlying asset (here, $AAPL). We want P = OP + RE to be neutral to small changes in the underlying price.

# Delta hedging a single option

In [71]:
def delta_hedge(call_option_prices):
    interest_rate = 0.05076 # 3-Month Treasury Yield
    volatility = 0.2 # initial guess ?
    option_type = 'call'
    stock_price = call_option_prices.iloc[0]['Underlying']
    option_price = call_option_prices.iloc[0]['C170']
    strike = 170
    time_to_maturity = call_option_prices.iloc[0]['T']

    # Initial stuff at t = 0
    bsm_obj = BSM(kind=option_type, S0=stock_price, K=strike, T=time_to_maturity, r=interest_rate, sigma=volatility)
    OP = bsm_obj.value() # Value of the option according to BSM
    RE = - bsm_obj.delta() * stock_price
    delta = bsm_obj.delta()
    
    print("Initial values:")
    print("Date:", call_option_prices.iloc[0]['Date'])
    print("Value of single call option according to BSM: ", round(OP))
    print("Value of the delta of the option according to BSM: ", round(delta, 2))
    print("Value of the long position according to BSM: ", round(OP))
    print("Value of the short position according to BSM: ", round(RE))
    print('\n')

    total_mean_squared_error = []
    
    # next we calculate changes in OP and RE and let A_0 = d OP - d RE
    # Hedge as long as there are dates in the dataframe
    for i in range(1, len(call_option_prices)-1, 1):
        new_bsm_obj = BSM(kind=option_type, S0=call_option_prices['Underlying'][i], K=strike, T=call_option_prices['T'][i], r=interest_rate, sigma=volatility)
        new_OP = new_bsm_obj.value()
        new_RE = - delta * call_option_prices['Underlying'][i]
        d_OP = new_OP - OP
        d_RE = delta*(call_option_prices['Underlying'][i+1] - call_option_prices['Underlying'][i])
        a_0 = d_OP - d_RE
        total_mean_squared_error.append(a_0**2)

        print('==================================================')
        print("Date:", call_option_prices['Date'][i])
        print("Value of a new single call option according to BSM: ", round(new_OP))
        print("Value of the delta of the option according to BSM: ", round(delta, 2))
        print("Value of the new long position according to BSM: ", round(new_OP))
        print("Value of the new short position according to BSM: ", round(new_RE))
        print("Difference of the long positions: ", round(OP - new_OP))
        print("Difference of the short positions: ", round(RE - new_RE))
        print("A_0: ", a_0)
        print('==================================================')
        print('\n')

        OP = new_OP
        RE = new_RE

        # Rehedging happens here
        # every second day, t2, t4, t6, so on..
        # Change the modulo for different intervals, 1 = every day, 30 = every month and so on
        if i % 2 == 0:
            delta = new_bsm_obj.delta()
            volatility = implied_volatility(price=OP, S=call_option_prices['Underlying'][i], K=strike, t=call_option_prices['T'][i], r=interest_rate, flag='c')
            print("Rehedging date:", call_option_prices['Date'][i])
            print("Delta updated, new delta is:", round(delta, 2))
            print("Volatility updated, new implied volatility is:", round(volatility, 2))
            print('To delta-hedge a single long call option, we need to readjust the RE:')
            RE = -delta*call_option_prices['Underlying'][i]
            print('Readjusting replicating portoflio RE, delta * underlying: ', round(RE))
            print('\n')
        
    # Count the total error
    total_error = 0
    for i in range(0, len(total_mean_squared_error)-1):
        total_error += total_mean_squared_error[i]
    total_error = total_error*(1/(len(total_mean_squared_error)-1))
    print(total_error)

In [72]:
delta_hedge(df)

Initial values:
Date: 2023-08-21
Value of single call option according to BSM:  17
Value of the delta of the option according to BSM:  0.69
Value of the long position according to BSM:  17
Value of the short position according to BSM:  -121


Date: 2023-08-22
Value of a new single call option according to BSM:  18
Value of the delta of the option according to BSM:  0.69
Value of the new long position according to BSM:  18
Value of the new short position according to BSM:  -122
Difference of the long positions:  -1
Difference of the short positions:  1
A_0:  -1.7621930857848378


Date: 2023-08-23
Value of a new single call option according to BSM:  21
Value of the delta of the option according to BSM:  0.69
Value of the new long position according to BSM:  21
Value of the new short position according to BSM:  -125
Difference of the long positions:  -3
Difference of the short positions:  3
A_0:  6.051357318220752


Rehedging date: 2023-08-23
Delta updated, new delta is: 0.75
Volatility u

# Delta-vega hedging a single option

In [70]:
def delta_vega_hedge(call_option_prices):
    interest_rate = 0.5076 # 3-Month Treasury Yield
    volatility = 0.2 # Initial guess?
    initial_stock_price = call_option_prices.iloc[0]['Underlying']
    strike_price = call_option_prices.iloc[0]['Strike']
    time_to_maturity = call_option_prices.iloc[0]['T']

    # Initial option pricing
    bsm_obj = BSM(S0=initial_stock_price, K=strike_price, T=time_to_maturity, r=interest_rate, sigma=volatility)
    initial_option_price = bsm_obj.call_price()
    initial_delta = bsm_obj.delta()
    initial_vega = bsm_obj.vega()

    # Hedging parameters
    delta_target = 0.5  # Target delta value
    vega_target = 0.1   # Target vega value

    # Lists to store results
    deltas = [initial_delta]
    vegas = [initial_vega]

    # Delta-vega hedging loop
    for i in range(1, len(call_option_prices)):
        # Update option pricing
        bsm_obj = BSM(S0=call_option_prices['Underlying'][i], K=strike_price, T=call_option_prices['T'][i], r=interest_rate, sigma=volatility)
        option_price = bsm_obj.call_price()
        delta = bsm_obj.delta()
        vega = bsm_obj.vega()

        # Calculate changes in delta and vega
        delta_change = delta - initial_delta
        vega_change = vega - initial_vega

        # Adjust stock position for delta and vega hedging
        stock_position = delta_target * delta_change + vega_target * vega_change

        # Store results
        deltas.append(delta)
        vegas.append(vega)

        # Update initial values for the next iteration
        initial_delta = delta
        initial_vega = vega

    # Create a DataFrame to store results
    result_df = pd.DataFrame({'Delta': deltas, 'Vega': vegas})

    return result_df

SyntaxError: incomplete input (92178917.py, line 2)

In [51]:
delta_vega_hedge(df)

kissa
